In [1]:
import numpy as np
import pyniggli
from opf_python.lat_type import lat_type
from opf_python.niggli_lat_id import niggli_id

In [2]:
def niggli_setting(A):
    """Finds the corrdinates in G6 for the lattice A. Where G6 is
    [A[:,0].A[:,0],A[:,1].A[:,1],A[:,2].A[:,2],
    2*A[:,1].A[:,2],2*A[:,2].A[:,0],2*A[:,0].A[:,1]]
    
    Args:
        A (numpy array): The lattice vectors as columns of a matrix.
        
    Returns:
        G (list): The coordinates on G6 space.
    """
    
    a = A[:,0]
    b = A[:,1]
    c = A[:,2]
    A = np.dot(a,a)
    B = np.dot(b,b)
    C = np.dot(c,c)
    xi = np.dot(b,c)
    eta = np.dot(a,c)
    zeta = np.dot(a,b)
    
    G = [A,B,C,xi,eta,zeta]
    return G

# The Idea

The idea here is to try and see if we can figure out a method to transform the symmetry preserving supercell from our basis to the users basis by using the niggli reduced cell. The workflow would be as follows:

1) identify which of the 44 niggli cells is in use (there are two triclinic cells included in those 44 which we'll disregard)

2) Find the Supercell in the niggli basis

3) Transform back to the users basis.

UC = N

S = NHCinv

# Face centered cubic

## First basis

Face centered cubic makes a good test case for the cubic cells, all cubic cells have a single unique niggli cell.

In [3]:
from fcc import fcc_srHNFs

In [4]:
U = np.transpose([[0.5,0,0.5],[0.5,0.5,0],[0,0,1]])
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

Using the brute force method I've now confirmed that this niggli cell has the same symmetry preserving HNFs as our original basis choice.

Now we'll verify that HNFs of different volume factors (VF) get transformed to the users basis properly.

### VF 4

In [5]:
H = fcc_srHNFs(4)[0]

In [6]:
S = np.dot(np.dot(N,H),np.linalg.inv(C))
lat_name(np.transpose(S))

('sc', array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]]), [0, 1, 2])

In [7]:
r = np.linalg.inv(np.transpose(U))
g = np.linalg.inv(np.transpose(S))
np.round(np.dot(np.linalg.inv(g),r),3)

array([[ 2.,  0.,  0.],
       [ 0.,  2.,  1.],
       [ 0.,  0.,  1.]])

Looks like the volume factor of 4 worked out.

### VF 16

In [8]:
H = fcc_srHNFs(16)[0]
S = np.dot(np.dot(N,H),np.linalg.inv(C))
lat_name(np.transpose(S))

('bcc', array([[-1,  1,  1],
        [ 1, -1,  1],
        [ 1,  1, -1]]), [0, 1, 2])

In [9]:
r = np.linalg.inv(np.transpose(U))
g = np.linalg.inv(np.transpose(S))
np.round(np.dot(np.linalg.inv(g),r),3)

array([[ 4., -0., -0.],
       [ 0.,  2.,  1.],
       [ 0.,  2.,  3.]])

So far so good. Now let's check one more case.

### VF 32

In [10]:
H = fcc_srHNFs(32)[0]
S = np.dot(np.dot(N,H),np.linalg.inv(C))
lat_name(np.transpose(S))

('sc', array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]]), [0, 1, 2])

In [11]:
r = np.linalg.inv(np.transpose(U))
g = np.linalg.inv(np.transpose(S))
np.round(np.dot(np.linalg.inv(g),r),3)

array([[ 4.,  0.,  0.],
       [ 0.,  4.,  2.],
       [ 0.,  0.,  2.]])

## Second FCC basis

Let's do a second basis just to be sure.

In [12]:
U = np.transpose([[1,1,2],[1,0,1],[2,1,1]])
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

### VF 4

In [13]:
H = fcc_srHNFs(4)[0]
S = np.dot(np.dot(N,H),np.linalg.inv(C))
lat_name(np.transpose(S))

('sc', array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]]), [0, 1, 2])

In [14]:
r = np.linalg.inv(np.transpose(U))
g = np.linalg.inv(np.transpose(S))
np.round(np.dot(np.linalg.inv(g),r),3)

array([[ 1.,  1.,  1.],
       [ 0.,  2.,  0.],
       [ 0.,  0.,  2.]])

### VF 16

In [15]:
H = fcc_srHNFs(16)[0]
S = np.dot(np.dot(N,H),np.linalg.inv(C))
lat_name(np.transpose(S))

('bcc', array([[-1,  1,  1],
        [ 1, -1,  1],
        [ 1,  1, -1]]), [0, 1, 2])

In [16]:
r = np.linalg.inv(np.transpose(U))
g = np.linalg.inv(np.transpose(S))
np.round(np.dot(np.linalg.inv(g),r),3)

array([[ 1.,  3.,  1.],
       [ 0.,  4.,  0.],
       [ 0.,  0.,  4.]])

### VF 32

In [17]:
H = fcc_srHNFs(32)[0]
S = np.dot(np.dot(N,H),np.linalg.inv(C))
lat_name(np.transpose(S))

('sc', array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]]), [0, 1, 2])

In [18]:
r = np.linalg.inv(np.transpose(U))
g = np.linalg.inv(np.transpose(S))
np.round(np.dot(np.linalg.inv(g),r),3)

array([[ 2.,  2.,  2.],
       [ 0.,  4.,  0.],
       [ 0.,  0.,  4.]])

## FCC rotated basis (10 degrees about the z-axis)

In [19]:
R = [[np.cos(10.*np.pi/180.),-np.sin(10.*np.pi/180.),0],
     [np.sin(10.*np.pi/180.),np.cos(10.*np.pi/180.),0],
    [0,0,1]]
U = np.dot(R,U)
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

### VF 4

In [21]:
H = fcc_srHNFs(4)[0]
S = np.dot(np.dot(N,H),np.linalg.inv(C))
lat_name(np.transpose(S))

('sc', array([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]]), [0, 1, 2])

In [22]:
r = np.linalg.inv(np.transpose(U))
g = np.linalg.inv(np.transpose(S))
temp =np.round(np.dot(np.linalg.inv(g),r),3)

array([[ 1.,  1.,  1.],
       [ 0.,  2.,  0.],
       [ 0.,  0.,  2.]])

In [29]:
np.allclose(temp%1,0)

True

### VF 16

In [30]:
H = fcc_srHNFs(16)[0]
S = np.dot(np.dot(N,H),np.linalg.inv(C))
print(lat_name(np.transpose(S))[0])
r = np.linalg.inv(np.transpose(U))
g = np.linalg.inv(np.transpose(S))
temp = np.round(np.dot(np.linalg.inv(g),r),3)
np.allclose(temp%1,0)

bcc


True

### VF 32

In [31]:
H = fcc_srHNFs(32)[0]
S = np.dot(np.dot(N,H),np.linalg.inv(C))
print(lat_name(np.transpose(S))[0])
r = np.linalg.inv(np.transpose(U))
g = np.linalg.inv(np.transpose(S))
temp = np.round(np.dot(np.linalg.inv(g),r),3)
np.allclose(temp%1,0)

sc


True

# Simple Tetragonal

Now we should check a second crystal choice. Simple tetragonal cell's are another where we already have the symmetry preserving HNFs for one of the two niggli cells. We'll have to restrict ourselves to the niggli setting with lattice norms of [A,A,C] where C > A for these tests. The following 4 lattices satisfy that condition.

In [3]:
from stet import stet_srHNFs

In [4]:
a1 = np.array([1,0,0])
a2 = np.array([0,1,0])
a3 = np.array([0,0,2])

As = [[a1+a2,a2,a3+a1],[a1+a2+a3,a2+a3,a1+a2],[a1,a2+a1,a3+a3],[-a1,-a2+a3,a3]]

## First Basis Choice

In [5]:
U = np.random.random()*np.transpose(As[0])
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

### VF 10

In [7]:
HNFs = stet_srHNFs(10)
crystal_fam = 4
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)


### VF 80

In [10]:
HNFs = stet_srHNFs(80)
crystal_fam = 4
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)


## Second basis choice

In [11]:
U = np.random.random()*np.transpose(As[1])
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

### VF 120

In [12]:
HNFs = stet_srHNFs(120)
crystal_fam = 4
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)


### VF 12

In [13]:
HNFs = stet_srHNFs(12)
crystal_fam = 4
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)


## Third Basis Choice (rotated by 15 degrees)

In [7]:
R = [[np.cos(15.*np.pi/180.),-np.sin(15.*np.pi/180.),0],
     [np.sin(15.*np.pi/180.),np.cos(15.*np.pi/180.),0],
    [0,0,1]]
U = np.random.random()*np.transpose(As[2])
U = np.dot(R,U)
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

### VF 9

In [8]:
HNFs = stet_srHNFs(9)
crystal_fam = 4
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)


### VF 125

In [9]:
HNFs = stet_srHNFs(125)
crystal_fam = 4
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)


## Fourth Basis Choice (rotated by 27 degrees)

In [5]:
R = [[np.cos(27.*np.pi/180.),-np.sin(27.*np.pi/180.),0],
     [np.sin(27.*np.pi/180.),np.cos(27.*np.pi/180.),0],
    [0,0,1]]
U = np.random.random()*np.transpose(As[3])
U = np.dot(R,U)
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

### VF 12

In [6]:
HNFs = stet_srHNFs(12)
crystal_fam = 4
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)


### VF 1000

In [7]:
HNFs = stet_srHNFs(1000)
crystal_fam = 4
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)
('Passed HNF number: ', 9)
('Passed HNF number: ', 10)
('Passed HNF number: ', 11)
('Passed HNF number: ', 12)
('Passed HNF number: ', 13)


# Simple Orthorhombic

In [3]:
from so import so_srHNFs

In [4]:
As = [[[1.00000000, 0.00000000, 0.00000000],
       [0.00000000, 1.73205080, 0.00000000],
       [0.00000000, 0.00000000, 3.26598640]],
      [[0.50000000, 0.50000000, 0.00000000],
      [0.00000000, 0.00000000, 1.00000000],
      [1.50000000, -1.50000000, 0.00000000]],
      [[0.50000000, 0.50000000, 0.00000000],
      [0.00000000, 0.00000000, 1.00000000],
      [2.00000000, -2.00000000, 0.00000000]]]
As = [np.transpose(i) for i in As]

## First basis choice

In [5]:
U = np.random.random()*As[0]
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

### VF 16

In [6]:
HNFs = so_srHNFs(16)
crystal_fam = 5
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S,eps_=1E-5)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)
('Passed HNF number: ', 9)
('Passed HNF number: ', 10)
('Passed HNF number: ', 11)
('Passed HNF number: ', 12)
('Passed HNF number: ', 13)
('Passed HNF number: ', 14)
('Passed HNF number: ', 15)
('Passed HNF number: ', 16)
('Passed HNF number: ', 17)
('Passed HNF number: ', 18)
('Passed HNF number: ', 19)
('Passed HNF number: ', 20)
('Passed HNF number: ', 21)
('Passed HNF number: ', 22)
('Passed HNF number: ', 23)
('Passed HNF number: ', 24)
('Passed HNF number: ', 25)
('Passed HNF number: ', 26)
('Passed HNF number: ', 27)
('Passed HNF number: ', 28)
('Passed HNF number: ', 29)
('Passed HNF number: ', 30)
('Passed HNF number: ', 31)
('Passed HNF number: ', 32)
('Passed HNF number: ', 33)
('Passed HNF number: ', 34)
('Passed HNF number: ', 35)
('

### VF 24

In [7]:
HNFs = so_srHNFs(24)
crystal_fam = 5
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S,eps_=1E-5)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)
('Passed HNF number: ', 9)
('Passed HNF number: ', 10)
('Passed HNF number: ', 11)
('Passed HNF number: ', 12)
('Passed HNF number: ', 13)
('Passed HNF number: ', 14)
('Passed HNF number: ', 15)
('Passed HNF number: ', 16)
('Passed HNF number: ', 17)
('Passed HNF number: ', 18)
('Passed HNF number: ', 19)
('Passed HNF number: ', 20)
('Passed HNF number: ', 21)
('Passed HNF number: ', 22)
('Passed HNF number: ', 23)
('Passed HNF number: ', 24)
('Passed HNF number: ', 25)
('Passed HNF number: ', 26)
('Passed HNF number: ', 27)
('Passed HNF number: ', 28)
('Passed HNF number: ', 29)
('Passed HNF number: ', 30)
('Passed HNF number: ', 31)
('Passed HNF number: ', 32)
('Passed HNF number: ', 33)
('Passed HNF number: ', 34)
('Passed HNF number: ', 35)
('

### VF 10

In [8]:
HNFs = so_srHNFs(10)
crystal_fam = 5
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S,eps_=1E-5)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)
('Passed HNF number: ', 9)
('Passed HNF number: ', 10)
('Passed HNF number: ', 11)
('Passed HNF number: ', 12)
('Passed HNF number: ', 13)
('Passed HNF number: ', 14)
('Passed HNF number: ', 15)
('Passed HNF number: ', 16)
('Passed HNF number: ', 17)
('Passed HNF number: ', 18)
('Passed HNF number: ', 19)
('Passed HNF number: ', 20)


## Second Basis Choice

In [9]:
U = np.random.random()*As[1]
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

### VF 18

In [10]:
HNFs = so_srHNFs(10)
crystal_fam = 5
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S,eps_=1E-5)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)
('Passed HNF number: ', 9)
('Passed HNF number: ', 10)
('Passed HNF number: ', 11)
('Passed HNF number: ', 12)
('Passed HNF number: ', 13)
('Passed HNF number: ', 14)
('Passed HNF number: ', 15)
('Passed HNF number: ', 16)
('Passed HNF number: ', 17)
('Passed HNF number: ', 18)
('Passed HNF number: ', 19)
('Passed HNF number: ', 20)


### VF 12

In [11]:
HNFs = so_srHNFs(12)
crystal_fam = 5
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S,eps_=1E-5)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)
('Passed HNF number: ', 9)
('Passed HNF number: ', 10)
('Passed HNF number: ', 11)
('Passed HNF number: ', 12)
('Passed HNF number: ', 13)
('Passed HNF number: ', 14)
('Passed HNF number: ', 15)
('Passed HNF number: ', 16)
('Passed HNF number: ', 17)
('Passed HNF number: ', 18)
('Passed HNF number: ', 19)
('Passed HNF number: ', 20)
('Passed HNF number: ', 21)
('Passed HNF number: ', 22)
('Passed HNF number: ', 23)
('Passed HNF number: ', 24)
('Passed HNF number: ', 25)
('Passed HNF number: ', 26)
('Passed HNF number: ', 27)
('Passed HNF number: ', 28)
('Passed HNF number: ', 29)
('Passed HNF number: ', 30)
('Passed HNF number: ', 31)
('Passed HNF number: ', 32)
('Passed HNF number: ', 33)
('Passed HNF number: ', 34)
('Passed HNF number: ', 35)
('

### VF 4

In [12]:
HNFs = so_srHNFs(4)
crystal_fam = 5
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S,eps_=1E-5)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)
('Passed HNF number: ', 9)
('Passed HNF number: ', 10)
('Passed HNF number: ', 11)
('Passed HNF number: ', 12)
('Passed HNF number: ', 13)
('Passed HNF number: ', 14)
('Passed HNF number: ', 15)
('Passed HNF number: ', 16)
('Passed HNF number: ', 17)
('Passed HNF number: ', 18)


## Third Basis Choice

In [13]:
U = np.random.random()*As[2]
B = pyniggli.reduced_cell(U)
N = B.niggli
C = B.C

### VF 12

In [14]:
HNFs = so_srHNFs(12)
crystal_fam = 5
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S,eps_=1E-5)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)
('Passed HNF number: ', 9)
('Passed HNF number: ', 10)
('Passed HNF number: ', 11)
('Passed HNF number: ', 12)
('Passed HNF number: ', 13)
('Passed HNF number: ', 14)
('Passed HNF number: ', 15)
('Passed HNF number: ', 16)
('Passed HNF number: ', 17)
('Passed HNF number: ', 18)
('Passed HNF number: ', 19)
('Passed HNF number: ', 20)
('Passed HNF number: ', 21)
('Passed HNF number: ', 22)
('Passed HNF number: ', 23)
('Passed HNF number: ', 24)
('Passed HNF number: ', 25)
('Passed HNF number: ', 26)
('Passed HNF number: ', 27)
('Passed HNF number: ', 28)
('Passed HNF number: ', 29)
('Passed HNF number: ', 30)
('Passed HNF number: ', 31)
('Passed HNF number: ', 32)
('Passed HNF number: ', 33)
('Passed HNF number: ', 34)
('Passed HNF number: ', 35)
('

### VF 8

In [15]:
HNFs = so_srHNFs(8)
crystal_fam = 5
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S,eps_=1E-5)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)
('Passed HNF number: ', 9)
('Passed HNF number: ', 10)
('Passed HNF number: ', 11)
('Passed HNF number: ', 12)
('Passed HNF number: ', 13)
('Passed HNF number: ', 14)
('Passed HNF number: ', 15)
('Passed HNF number: ', 16)
('Passed HNF number: ', 17)
('Passed HNF number: ', 18)
('Passed HNF number: ', 19)
('Passed HNF number: ', 20)
('Passed HNF number: ', 21)
('Passed HNF number: ', 22)
('Passed HNF number: ', 23)
('Passed HNF number: ', 24)
('Passed HNF number: ', 25)
('Passed HNF number: ', 26)
('Passed HNF number: ', 27)
('Passed HNF number: ', 28)
('Passed HNF number: ', 29)
('Passed HNF number: ', 30)
('Passed HNF number: ', 31)
('Passed HNF number: ', 32)
('Passed HNF number: ', 33)
('Passed HNF number: ', 34)
('Passed HNF number: ', 35)
('

### VF 14

In [16]:
HNFs = so_srHNFs(12)
crystal_fam = 5
for i in range(len(HNFs)):
    H = HNFs[i]
    S = np.dot(np.dot(N,H),np.linalg.inv(C))
    lat_name, niggli_n, lat_fam = niggli_id(S,eps_=1E-5)
    r = np.linalg.inv(np.transpose(U))
    g = np.linalg.inv(np.transpose(S))
    temp = np.round(np.dot(np.linalg.inv(g),r),3)
    if lat_fam > crystal_fam or not np.allclose(temp%1,0):
        print("FAILED on HNF number: ",i)
    else:
        print("Passed HNF number: ",i)

('Passed HNF number: ', 0)
('Passed HNF number: ', 1)
('Passed HNF number: ', 2)
('Passed HNF number: ', 3)
('Passed HNF number: ', 4)
('Passed HNF number: ', 5)
('Passed HNF number: ', 6)
('Passed HNF number: ', 7)
('Passed HNF number: ', 8)
('Passed HNF number: ', 9)
('Passed HNF number: ', 10)
('Passed HNF number: ', 11)
('Passed HNF number: ', 12)
('Passed HNF number: ', 13)
('Passed HNF number: ', 14)
('Passed HNF number: ', 15)
('Passed HNF number: ', 16)
('Passed HNF number: ', 17)
('Passed HNF number: ', 18)
('Passed HNF number: ', 19)
('Passed HNF number: ', 20)
('Passed HNF number: ', 21)
('Passed HNF number: ', 22)
('Passed HNF number: ', 23)
('Passed HNF number: ', 24)
('Passed HNF number: ', 25)
('Passed HNF number: ', 26)
('Passed HNF number: ', 27)
('Passed HNF number: ', 28)
('Passed HNF number: ', 29)
('Passed HNF number: ', 30)
('Passed HNF number: ', 31)
('Passed HNF number: ', 32)
('Passed HNF number: ', 33)
('Passed HNF number: ', 34)
('Passed HNF number: ', 35)
('

# Conclusion

The tests so far indicate that the transformation to the Niggli cell will work. However there are still 39 cases to check and verify in addition to a more detailed study even of these systems.

What I need to do is find a way to quickly identify the lattice type, this can be accomplished via niggli faster than the current method, then verify that the supercell's retain symmetry for a larger test case.

In other words I need a far more extensive testing suite that gets wrapped into a subroutine so that it will automatically generate supercells for multiple densities for a given cell and for each supercell verify that the cell has equal or greater symmetry than the parent. If ever this isn't the case then we'll need to have it stop and report the error, i.e., let us know that the method doesn't actually work.